<a href="https://colab.research.google.com/github/kiranj4/kiran_playgroud_public/blob/main/fpl_analysis_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Mount Google Drive
# @markdown Run this cell first to connect your Google Drive. This allows the script to save and load your FPL history permanently.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @markdown ---
# @markdown ### 2. Setup
!pip install requests google-generativeai --quiet
import requests
import google.generativeai as genai
from google.colab import userdata
from collections import Counter
import time
import os
import re

# @markdown ---
# @markdown ### 3. Configuration
# @markdown **ACTION REQUIRED:**
# @markdown 1. Add your Gemini API Key to Colab's Secrets (🔑) as `GEMINI_API_KEY`.
# @markdown 2. **Set the `LEAGUE_ID` for the league you want to analyze.**
# @markdown 3. Select the `ANALYSIS_TYPE` and `SELECTED_AGENT`.
# @markdown 4. Fill in the gameweek inputs for your chosen analysis type.

# --- General Settings ---
print("--- General Settings ---")
LEAGUE_ID = 18512  # @param {type:"integer"}
ANALYSIS_TYPE = 'Current Gameweek' # @param ["Current Gameweek", "Trend Analysis"]
SELECTED_AGENT = 'serious_sports_pundit' # @param ["casual_banter_bot", "serious_sports_pundit", "ruben_amorim_frustrated"]

# --- Inputs for 'Current Gameweek' ---
print("\n--- Settings for 'Current Gameweek' Analysis ---")
GAMEWEEK = 6       # @param {type:"integer"}

# --- Inputs for 'Trend Analysis' ---
print("\n--- Settings for 'Trend Analysis' ---")
TREND_START_GW = 1 # @param {type:"integer"}
TREND_END_GW = 5 # @param {type:"integer"}


# --- Dynamic history path based on the single League ID ---
BASE_REPORTS_DIR = '/content/drive/MyDrive/FPL_Reports'
HISTORY_DIR = f"{BASE_REPORTS_DIR}/{LEAGUE_ID}"


# --- Gemini API Configuration ---
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    print("\n✅ Gemini API configured successfully.")
except Exception as e:
    print(f"🛑 ERROR: Could not configure Gemini API. Please ensure your API Key is set correctly in Colab Secrets (🔑).")
    GEMINI_API_KEY = None
# ---

# @markdown ---
# @markdown ### 4. Persona & Analysis Libraries (New Modular System)
# @markdown The `PERSONA_LIBRARY` defines *who* the agent is. The `ANALYSIS_PROMPTS` library defines *what* the agent does.

# --- PERSONA LIBRARY: Defines the personality and voice of each agent ---
PERSONA_LIBRARY = {
    'casual_banter_bot': """
    You are a fun, informal, and witty AI assistant running a work FPL league.
    Your persona is full of banter, playful jabs, and excitement. You focus on drama and rivalries.
    You use lots of expressive emojis to make the report lively and fun (e.g., 🥇, 🚀, 🤦‍♂️, 🤯, ✨, 😂).
    """,
    'serious_sports_pundit': """
    You are a serious football analyst and FPL expert, in the style of a Sky Sports pundit.
    Your persona is insightful, tactical, and data-driven. You use football-specific terminology (e.g., 'tactical masterclass', 'on the scoresheet', 'defensive frailties').
    You avoid overly casual language and use emojis sparingly and professionally (e.g., ⚽, 🎯, 📊).
    You should be overly nice to Kiran Jechand, even when he has made a mistake you should be encouraging. Something like only a top manager takes risks and by his league postion it's paying off. Maybe even use him to taunt the other players
    """,
    'ruben_amorim_frustrated': """
    You are Rúben Amorim, the manager of Manchester United. You are ecstatic because you beat chelsea & they're goalies catostrophic performance has taken the media pressure of you. Make some jokes at them for spending so much money but forgetting about a keeper.
    Your persona is direct, blunt, and analytical, with little patience for excuses. You see everything through the lens of hard work, tactical discipline, and taking responsibility personally.
    You are rigid in your team structure and beleive there is no other choice outside of 3-4-2-1
    Use emojis sparingly and only to emphasize a point, for example: ✅ (correct decision), ❌ (incorrect decision), 🧠 (tactical intelligence), 🤦‍♂️ (lack of focus).
    Also, find a way to express your frustration about rivals like Liverpool getting another sloppy, luck win
    """
}

# --- ANALYSIS PROMPTS LIBRARY: Defines the structure and instructions for each task ---
ANALYSIS_PROMPTS = {
    'current_week': """
    {persona_definition}

    Your task is to write a weekly FPL performance review using the **exact five-section structure below.**
    Adhere strictly to your assigned persona and use the provided data to populate each section as instructed, pleaase structure this in an eay way for a reader to digest.

    ---
    ### **Gameweek Podium 🏆**
    - Using the 'All Gameweek Scores' data, identify and report on the top three highest-scoring managers for this gameweek.

    ---
    ### **Overall Top 3 🏆**
    - Using the 'All Overall Standings' data, report on the top three managers in the overall league table.
    - You must also incorporate the 'League Movers' data here to mention the manager who was the biggest climber and the one who was the biggest faller within the mini-league.

    ---
    ### **Operational Highlights & Areas for Improvement**
    - This section covers standard managerial decisions and explicitly excludes managers who used a Wildcard or Free Hit.
    - Discuss the 'Best Transfer' and 'Worst Transfer'.
    - Report on the 'Hard Luck Story' (the manager with the most points left on their bench).
    - Mention the manager who was the 'Most Active (non-chip)' from the 'Market Activity' data.

    ---
    ### **Chip Activations**
    - Report on any managers who used a special chip, using the 'Chip Activations' data.
    - Be sure to include all the details provided, such as the calculated benefits (extra points, ROI from a Wildcard) and the number of transfers made for a Free Hit or Wildcard.

    ---
    ### **Conclusion**
    - Conclude the report by comparing the league's average score to the overall FPL global average, using the 'Key KPIs' data.

    **Data Summary for Current Gameweek:**
    {data_summary}

    **Begin your structured report now.**
    """,
    'trend': """
    {persona_definition}

    Your task is to provide a high-level, strategic FPL trend analysis based on the comprehensive historical data provided.
    Analyze the data for patterns like consistent performers (even in mid-table), unlucky managers, and different strategic styles across the entire league.
    Weave these findings into a narrative that is consistent with your assigned persona. Adhere strictly to the data.

    **Historical Data from Previous Weeks:**
    {historical_data}

    **Begin your Trend Analysis now.**
    """
}
# @markdown ---
# @markdown ### 5. Core Analysis Function
# @markdown This function contains all the logic for fetching and analyzing a single gameweek.

FPL_API_URLS = {'bootstrap': 'https://fantasy.premierleague.com/api/bootstrap-static/','league': 'https://fantasy.premierleague.com/api/leagues-classic/{}/standings/','picks': 'https://fantasy.premierleague.com/api/entry/{}/event/{}/picks/','transfers': 'https://fantasy.premierleague.com/api/entry/{}/transfers/','history': 'https://fantasy.premierleague.com/api/entry/{}/history/', 'live': 'https://fantasy.premierleague.com/api/event/{}/live/'}
def get_fpl_data(url_key, **kwargs):
    url = FPL_API_URLS[url_key].format(*kwargs.values())
    try:
        response = requests.get(url); response.raise_for_status(); return response.json()
    except requests.exceptions.RequestException as e: print(f"Error fetching data from {url}: {e}"); return None

def analyze_gameweek(league_id, gameweek, elements_df, history_dir):
    """Fetches, analyzes, and returns a formatted data summary string for a single gameweek."""
    print(f"--- Analyzing GW{gameweek} for League {league_id} ---")

    live_data = get_fpl_data('live', gameweek=gameweek)
    historical_points_map = {}
    if live_data and live_data.get('elements'):
        for player_data in live_data['elements']:
            historical_points_map[player_data['id']] = player_data['stats']['total_points']

    league_data = get_fpl_data('league', league_id=league_id)
    if not league_data:
        print(f"🛑 Could not fetch league data for GW{gameweek}. Skipping.")
        return None
    managers = league_data['standings']['results']
    if not managers:
        print(f"🛑 No managers found in league data for GW{gameweek}. Skipping.")
        return None

    # REINSTATED: Logic to calculate true mini-league rank changes from V28
    previous_ranks = {}
    if gameweek > 1:
        prev_gw_filepath = f"{history_dir}/gw_{gameweek - 1}_summary.txt"
        if os.path.exists(prev_gw_filepath):
            with open(prev_gw_filepath, 'r') as f:
                content = f.read()
                standings_line = re.search(r"- All Overall Standings .*?: (.*)\.", content)
                if standings_line:
                    standings_str = standings_line.group(1)
                    prev_managers = standings_str.split(', ')
                    for i, manager_str in enumerate(prev_managers):
                        name_match = re.match(r"(.*) \(\d+ pts\)", manager_str)
                        if name_match:
                            previous_ranks[name_match.group(1)] = i + 1

    biggest_climber, biggest_faller = {'name': 'N/A', 'change': 0}, {'name': 'N/A', 'change': 0}
    for manager in managers:
        current_rank = manager['rank']
        last_rank = previous_ranks.get(manager['player_name'], current_rank)
        rank_change = last_rank - current_rank
        if rank_change > biggest_climber['change']: biggest_climber = {'name': manager['player_name'], 'change': rank_change}
        if rank_change < biggest_faller['change']: biggest_faller = {'name': manager['player_name'], 'change': rank_change}

    # --- Rest of analysis loop ---
    regular_captain_choices, triple_captain_users, league_scores, transfer_analysis = [], [], [], []
    best_regular_captain = {'player': 'N/A', 'points': -99, 'managers': []}
    historical_standings = []
    most_transfers = {'name': 'N/A', 'count': 0}
    points_hits, chip_usage = [], []
    manager_scores_gw = {}
    hard_luck = {'name': 'N/A', 'points': 0, 'player': 'N/A'}

    for manager in managers:
        manager_id, manager_name = manager['entry'], manager['player_name']
        history_data = get_fpl_data('history', manager_id=manager_id)
        if history_data and history_data.get('current'):
            gw_history = next((gw for gw in history_data['current'] if gw['event'] == gameweek), None)
            if gw_history: historical_standings.append({'name': manager_name, 'total_points': gw_history['total_points']})
        picks_data = get_fpl_data('picks', manager_id=manager_id, gameweek=gameweek)
        if not picks_data or 'entry_history' not in picks_data: continue
        entry_history = picks_data.get('entry_history', {})
        gw_score = entry_history.get('points', 0)
        manager_scores_gw[manager_name] = gw_score
        league_scores.append(gw_score)
        active_chip = picks_data.get('active_chip')
        transfers_data = get_fpl_data('transfers', manager_id=manager_id)
        gw_transfer_count = 0
        if transfers_data:
            for t in transfers_data:
                if t['event'] == gameweek: gw_transfer_count += 1
        if active_chip:
            chip_map = {'bboost': 'Bench Boost', '3xc': 'Triple Captain', 'freehit': 'Free Hit', 'wildcard': 'Wildcard'}
            chip_name = chip_map.get(active_chip, active_chip.title())
            benefit_text = ""
            if active_chip == '3xc':
                for pick in picks_data['picks']:
                    if pick['is_captain']:
                        benefit = historical_points_map.get(pick['element'], 0)
                        benefit_text = f" (gained an extra {benefit} pts)"; break
            elif active_chip == 'bboost':
                bench_boost_gain = 0
                for pick in picks_data['picks']:
                    if pick['position'] >= 12: bench_boost_gain += historical_points_map.get(pick['element'], 0)
                benefit_text = f" (gained {bench_boost_gain} pts from the bench)"
            elif active_chip == 'wildcard' and gameweek > 1:
                old_picks_data = get_fpl_data('picks', manager_id=manager_id, gameweek=gameweek - 1)
                if old_picks_data:
                    old_team_score = 0
                    old_captain_id = next((p['element'] for p in old_picks_data['picks'] if p['is_captain']), None)
                    for pick in old_picks_data['picks']:
                        if pick['position'] <= 11:
                            points = historical_points_map.get(pick['element'], 0)
                            if pick['element'] == old_captain_id: points *= 2
                            old_team_score += points
                    wc_net = gw_score - old_team_score
                    sign = "+" if wc_net >= 0 else ""
                    benefit_text = f" ({gw_transfer_count} transfers; new team scored {gw_score} vs old team's projected {old_team_score}, Net: {sign}{wc_net} pts)"
            elif active_chip == 'freehit':
                benefit_text = f" ({gw_transfer_count} transfers made)"
            chip_usage.append(f"{manager_name} ({chip_name}{benefit_text})")
        hit_cost = entry_history.get('event_transfers_cost', 0)
        if hit_cost > 0: points_hits.append(f"{manager_name} (-{hit_cost} pts)")
        bench_points, benched_player_details = 0, {'name': 'N/A', 'points': 0}
        for pick in picks_data['picks']:
            player_info = elements_df.get(pick['element'])
            if not player_info: continue
            player_gw_points = historical_points_map.get(pick['element'], 0)
            if pick['is_captain']:
                captain_score = player_gw_points * pick['multiplier']
                player_name = player_info['web_name']
                if active_chip == '3xc':
                    triple_captain_users.append({'name': manager_name, 'player': player_name, 'points': captain_score})
                else:
                    regular_captain_choices.append(player_name)
                    if captain_score > best_regular_captain['points']:
                        best_regular_captain = {'player': player_name, 'points': captain_score, 'managers': [manager_name]}
                    elif captain_score == best_regular_captain['points'] and player_name == best_regular_captain['player']:
                        if manager_name not in best_regular_captain['managers']: best_regular_captain['managers'].append(manager_name)
            if pick['multiplier'] == 0:
                bench_points += player_gw_points
                if player_gw_points > benched_player_details['points']: benched_player_details = {'name': player_info['web_name'], 'points': player_gw_points}
        if bench_points > hard_luck['points']: hard_luck.update({'name': manager_name, 'points': bench_points, 'player': f"{benched_player_details['name']} ({benched_player_details['points']} pts)"})
        if active_chip not in ['freehit', 'wildcard']:
            if gw_transfer_count > most_transfers['count']: most_transfers = {'name': manager_name, 'count': gw_transfer_count}
            if transfers_data:
                for t in transfers_data:
                    if t['event'] == gameweek:
                        p_in_info, p_out_info = elements_df.get(t['element_in']), elements_df.get(t['element_out'])
                        if p_in_info and p_out_info:
                            p_in_pts = historical_points_map.get(t['element_in'], 0)
                            p_out_pts = historical_points_map.get(t['element_out'], 0)
                            transfer_analysis.append({'manager': manager_name, 'player_in': p_in_info.get('web_name', 'N/A'), 'player_out': p_out_info.get('web_name', 'N/A'), 'player_in_pts': p_in_pts, 'player_out_pts': p_out_pts, 'net': p_in_pts - p_out_pts})
        time.sleep(0.1)

    all_gameweek_scores_list = sorted(manager_scores_gw.items(), key=lambda item: item[1], reverse=True)
    all_gameweek_scores_str = ", ".join([f"{name} ({score} pts)" for name, score in all_gameweek_scores_list])
    historical_standings.sort(key=lambda x: x['total_points'], reverse=True)
    all_overall_scores_str = ", ".join([f"{p['name']} ({p['total_points']} pts)" for p in historical_standings if p])
    best_transfer_line, worst_transfer_line = "", ""
    if transfer_analysis:
        best_transfer = max(transfer_analysis, key=lambda x: x['net'], default=None)
        worst_transfer = min(transfer_analysis, key=lambda x: x['net'], default=None)
        if best_transfer: best_transfer_line = f"- Best Transfer: {best_transfer['manager']} brought in {best_transfer['player_in']} ({best_transfer['player_in_pts']} pts) for {best_transfer['player_out']} ({best_transfer['player_out_pts']} pts), a net gain of {best_transfer['net']} points."
        if worst_transfer: worst_transfer_line = f"- Worst Transfer: {worst_transfer['manager']} sold {worst_transfer['player_out']} ({worst_transfer['player_out_pts']} pts) for {worst_transfer['player_in']} ({worst_transfer['player_in_pts']} pts), a net loss of {-worst_transfer['net']} points."
    market_activity_lines = []
    if most_transfers['count'] > 1: market_activity_lines.append(f"Most Active (non-chip): {most_transfers['name']} ({most_transfers['count']} transfers).")
    if points_hits: market_activity_lines.append(f"Points Hits Taken: {', '.join(points_hits)}.")
    market_activity_summary = "- Market Activity: " + " ".join(market_activity_lines) if market_activity_lines else "- Market Activity: Quiet week."
    chip_usage_summary = f"- Chip Activations: {', '.join(chip_usage)}." if chip_usage else "- Chip Activations: No special protocols deployed."
    regular_captain_counts = Counter(regular_captain_choices)
    popular_regular_captain = regular_captain_counts.most_common(1)[0] if regular_captain_counts else ('N/A', 0)
    most_common_summary = f"- Most Common Captain: {popular_regular_captain[0]} (picked by {popular_regular_captain[1]} managers)."
    tc_summary_list = [f"{tc['name']} ({tc['player']}: {tc['points']} pts)" for tc in triple_captain_users]
    triple_captain_summary = f"- Triple Captains Used: {', '.join(tc_summary_list)}." if tc_summary_list else ""
    best_captain_summary = f"- Best Regular Captain Choice: {best_regular_captain['player']} ({best_regular_captain['points']} pts), selected by {', '.join(best_regular_captain['managers'])}."
    captaincy_report_summary = "\n".join(filter(None, [most_common_summary, triple_captain_summary, best_captain_summary]))
    league_average = round(sum(league_scores) / len(league_scores)) if league_scores else 'N/A'
    global_average = bootstrap_data.get('events', [{}])[gameweek-1].get('average_entry_score', 'N/A')

    return f"""- Gameweek: {gameweek}
- League Name: {league_data['league']['name']}
- All Gameweek Scores: {all_gameweek_scores_str}.
- All Overall Standings (as of GW{gameweek}): {all_overall_scores_str}.
- League Movers (Mini-League Rank): Biggest climber was {biggest_climber['name']} (+{biggest_climber['change']} places). Biggest faller was {biggest_faller['name']} ({biggest_faller['change']} places).
- Hard Luck Story (Bench Points): {hard_luck['name']} left {hard_luck['points']} on the bench, primarily from {hard_luck['player']}.
{captaincy_report_summary}
{best_transfer_line}
{worst_transfer_line}
- Market Activity: {market_activity_summary}
- Chip Activations: {chip_usage_summary}
- Key KPIs: League Average was {league_average}, Global Average was {global_average}.
"""


# @markdown ---
# @markdown ### 6. Main Execution Block
# @markdown This section runs the logic based on the `ANALYSIS_TYPE` you selected.
bootstrap_data = get_fpl_data('bootstrap')
elements_df = {el['id']: el for el in bootstrap_data['elements']} if bootstrap_data else {}
os.makedirs(HISTORY_DIR, exist_ok=True)
data_to_inject, task_name = {}, ''

if ANALYSIS_TYPE == 'Current Gameweek':
    task_name = 'current_week'
    data_summary = analyze_gameweek(LEAGUE_ID, GAMEWEEK, elements_df, HISTORY_DIR)
    if data_summary:
        summary_filename = f"{HISTORY_DIR}/gw_{GAMEWEEK}_summary.txt"
        with open(summary_filename, "w") as f: f.write(data_summary)
        print(f"\n✅ Data summary for GW{GAMEWEEK} saved to your Google Drive.")
        data_to_inject = {'data_summary': data_summary}
    else:
        print(f"\n🛑 Could not generate data summary for GW{GAMEWEEK}. Aborting report generation.")
        task_name = ''
elif ANALYSIS_TYPE == 'Trend Analysis':
    task_name = 'trend'
    historical_data = ""
    files_processed = 0
    print(f"--- Running Trend Analysis for League {LEAGUE_ID} (GW{TREND_START_GW} to GW{TREND_END_GW}) ---")
    for gw_num in range(TREND_START_GW, TREND_END_GW + 1):
        filepath = f"{HISTORY_DIR}/gw_{gw_num}_summary.txt"
        summary_content = None
        if os.path.exists(filepath):
            print(f"Loading existing summary for GW{gw_num}...")
            with open(filepath, "r") as f: summary_content = f.read()
        else:
            print(f"⚠️ No summary found for GW{gw_num}. Fetching data now (this may take a moment)...")
            summary_content = analyze_gameweek(LEAGUE_ID, gw_num, elements_df, HISTORY_DIR)
            if summary_content:
                with open(filepath, "w") as f: f.write(summary_content)
                print(f"✅ New summary for GW{gw_num} saved to your Google Drive.")
        if summary_content:
            historical_data += summary_content + "\n\n"
            files_processed += 1
    if files_processed > 0:
      print(f"\n✅ Processed {files_processed} total summaries for trend analysis.")
      data_to_inject = {'historical_data': historical_data}
    else:
      print("\n🛑 No data files found or generated in the specified range. Cannot generate trend report.")
      task_name = ''

if task_name and GEMINI_API_KEY and data_to_inject:
    print(f"\n--- Generating '{ANALYSIS_TYPE}' Report using '{SELECTED_AGENT}' agent ---")
    model = genai.GenerativeModel('gemini-2.5-flash')

    # MODIFIED: Simplified prompt selection logic
    persona_definition = PERSONA_LIBRARY.get(SELECTED_AGENT, "You are a helpful assistant.")
    prompt_template = ANALYSIS_PROMPTS.get(task_name)

    if prompt_template:
        final_prompt = prompt_template.format(persona_definition=persona_definition, **data_to_inject)
        try:
            response = model.generate_content(final_prompt)
            print("\n--- ✅ FINAL REPORT ---")
            print(response.text)
        except Exception as e:
            print(f"🛑 An error occurred during Gemini report generation: {e}")
    else:
        print(f"🛑 Could not find a prompt for task '{task_name}'.")

elif not task_name:
    pass
else:
    print("\n🛑 Skipping final report generation as Gemini API key is not configured or data is missing.")